In [1]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, Conv1D
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [2]:
cd "/content/drive/My Drive/Colab Notebooks/dw_mtrx/matrix_three/dw_mtrx3"

/content/drive/My Drive/Colab Notebooks/dw_mtrx/matrix_three/dw_mtrx3


In [3]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [4]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [5]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
  )
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [6]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])

#get_cnn_v5(input_shape, num_classes).summary()

In [7]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
  1/272 [..............................] - ETA: 0s - loss: 26.3990 - accuracy: 0.0156WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
272/272 [==============================] - 12s 45ms/step - loss: 2.7334 - accuracy: 0.2651 - val_loss: 1.4240 - val_accuracy: 0.5256
Epoch 2/5
272/272 [==============================] - 11s 42ms/step - loss: 1.1469 - accuracy: 0.6294 - val_loss: 0.4532 - val_accuracy: 0.8578
Epoch 3/5
272/272 [==============================] - 11s 42ms/step - loss: 0.5441 - accuracy: 0.8250 - val_loss: 0.1268 - val_accuracy: 0.9606
Epoch 4/5
272/272 [==============================] - 11s 42ms/step - loss: 0.3435 - accuracy: 0.8954 - val_loss: 0.0620 - val_accuracy: 0.9816
Epoch 5/5
272/272 [==============================] - 11s 42ms/ste

0.9612244897959183

In [12]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')
  ])

In [13]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [14]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0, 1),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0, 1),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 120.0, 'dropout_cnn_block_one': 0.5325006871295006, 'dropout_cnn_block_three': 0.430823998127576, 'dropout_cnn_block_two': 0.7069283296995881, 'dropout_dense_block_one': 0.8927032160584272, 'dropout_dense_block_two': 0.032959253595548854}
accuracy=0.30612245202064514
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.8241398230957008, 'dropout_cnn_block_three': 0.3340059690738738, 'dropout_cnn_block_two': 0.14489781214108888, 'dropout_dense_block_one': 0.5121571940924496, 'dropout_dense_block_two': 0.41964721391726}
accuracy=0.5814058780670166
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.8547937921793823, 'dropout_cnn_block_three': 0.19748669487311699, 'dropout_cnn_block_two': 0.09746086261294828, 'dropout_dense_block_one': 0.345840513345136, 'dropout_dense_block_two': 0.18977064765037788}
accuracy=0.19682539999485016
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.21578738231112216, 'dropout_cnn_block_three': 0.16112852034524083, 'dropout_cnn_block_two': 0.4470705074